In [ ]:
import sys
sys.path.append("/nydl/code/open-metric-learning/")

In [ ]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import yaml
from pathlib import Path
from IPython.core.display import HTML
from IPython.display import display
import torch
import random
import matplotlib.pyplot as plt

from oml.lightning.entrypoints.validate import pl_val
from oml.lightning.callbacks.metric import MetricValCallback
from oml.const import MOCK_DATASET_PATH
from oml.utils.download_mock_dataset import download_mock_dataset

display(HTML("<style>.container { width:100% !important; }</style>"))
pd.set_option('display.max_rows', 330)

%matplotlib inline

In [ ]:
#weights: /home/daloro/python_projects/open-metric-learning/embedder.ckpt
#/home/daloro/python_projects/open-metric-learning/postprocessor.ckpt

In [ ]:
cfg = f"""
    dataset_root: /home/daloro/data/DeepFashion_InShop/
    dataframe_name: df.csv
    logs_root: /home/daloro/logs
    bs_val: 224
    num_workers: 12
    
    transforms_val:
      name: norm_resize_hypvit_torch
      args:
        im_size: 224
        crop_size: 224

    model:
      name: vit
      args:
        arch: vits16
        normalise_features: False
        use_multi_scale: False
        weights: /home/daloro/Downloads/embedder_inshop.ckpt

    metric_args:
      cmc_top_k: [1, 5, 10]
      map_top_k: [5, 10]
      return_only_main_category: True
      
      
    accelerator: gpu
    devices: 1
  
"""

In [ ]:
trainer, ret_dict = pl_val(yaml.load(cfg, Loader=yaml.Loader));
clb_metric = [x for x in trainer.callbacks if isinstance(x, MetricValCallback)][0]

In [ ]:
clb_metric = [x for x in trainer.callbacks if isinstance(x, MetricValCallback)][0]

In [ ]:
y = torch.load("/home/daloro/Downloads/post_inshop.ckpt", map_location="cpu")["state_dict"]

kvs = list(y.items())
for k, v in kvs:
    if k.startswith("model_pairwise"):
        del y[k]
        
torch.save({"state_dict": y}, "/home/daloro/Downloads/post_inshop.ckpt")

In [ ]:
cfg_p =  cfg + f"""
    postprocessor:
      name: pairwise_images
      args:
        top_n: 9
        pairwise_model:
          name: concat_siamese
          args:
            mlp_hidden_dims: [192]
            weights: /home/daloro/Downloads/post_inshop.ckpt
            extractor:
              name: vit
              args:
                arch: vits16
                normalise_features: False
                use_multi_scale: False
                weights: null
        transforms:
          name: norm_resize_hypvit_torch
          args:
            im_size: 224
            crop_size: 224
        num_workers: 12
        batch_size: 224
        verbose: True

"""

In [ ]:
import time
st = time.perf_counter()
trainer_p, ret_dict_p = pl_val(yaml.load(cfg_p, Loader=yaml.Loader));
et = time.perf_counter()
clb_metric_p = [x for x in trainer_p.callbacks if isinstance(x, MetricValCallback)][0]

In [ ]:
(et - st) / 60

In [ ]:
(60 * 3 + 8) / (5 * len(clb_metric_p.metric.metrics_unreduced["OVERALL"]["cmc"][1]))

In [ ]:
clb_metric_p = [x for x in trainer_p.callbacks if isinstance(x, MetricValCallback)][0]

In [ ]:
cmc_1 = clb_metric.metric.metrics_unreduced["OVERALL"]["cmc"][1]
cmc_1_p = clb_metric_p.metric.metrics_unreduced["OVERALL"]["cmc"][1]

In [ ]:
ids = random.sample(torch.nonzero(cmc_1_p > cmc_1).squeeze().tolist(), 10)

for idx in ids:
    print(idx)
    fig = clb_metric.metric.get_plot_for_queries([idx], n_instances=5, verbose=True)
    plt.show()
    fig = clb_metric_p.metric.get_plot_for_queries([idx], n_instances=5, verbose=True)
    plt.show()


In [ ]:
from oml.models.siamese import ConcatSiamese
from oml.models.vit.vit import ViTExtractor

extractor = ViTExtractor(arch="vits16", normalise_features=False, weights='/home/daloro/python_projects/open-metric-learning/embedder.ckpt')
siamese = ConcatSiamese(extractor=extractor,
              mlp_hidden_dims=[192],
              weights="/home/daloro/python_projects/open-metric-learning/postprocessor.ckpt"
             )

In [ ]:
idx = 6478

print(idx)
fig = clb_metric.metric.plow_with_att([idx], extractor=extractor, siamese=siamese, prefix='embedder', save_dir='/home/daloro/paper_images', n_instances=5, verbose=True)
plt.show()
fig = clb_metric_p.metric.plow_with_att([idx], extractor=extractor, siamese=siamese, prefix='postprocessor', save_dir='/home/daloro/paper_images', n_instances=5, verbose=True)
plt.show()

In [ ]:
idx = 14062

ids_to_save = [14062, 13976, 488, 12196, 4216, 4392, 6668, 3268, 2431]

for idx in ids_to_save:
    f = clb_metric.metric.plow_with_att([idx], extractor=extractor, siamese=siamese, prefix='embedder', save_dir='/home/daloro/paper_images', n_instances=5, verbose=True);
    f= clb_metric_p.metric.plow_with_att([idx], extractor=extractor, siamese=siamese, prefix='postprocessor', save_dir='/home/daloro/paper_images', n_instances=5, verbose=True);